<a href="https://colab.research.google.com/github/Wisnih/NBA-players-value/blob/main/NBA_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
pd.set_option('display.max_columns', None)
import time
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from fuzzywuzzy import fuzz

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Scraping

In [ ]:
test_url ='https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season=2023-24&SeasonType=Regular%20Season&StatCategory=PTS'

In [ ]:
r = requests.get(url = test_url).json()

In [ ]:
table_headers = r['resultSet']['headers']

In [ ]:
pd.DataFrame(r['resultSet']['rowSet'], columns=table_headers)

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,1629029,1,Luka Doncic,1610612742,DAL,58,2168,678,1373,0.494,227,600,0.378,408,518,0.788,45,475,520,571,83,33,230,119,1991,2163,2.48,0.36
1,1628983,2,Shai Gilgeous-Alexander,1610612760,OKC,64,2202,704,1294,0.544,85,225,0.378,497,564,0.881,55,303,358,411,133,61,139,163,1990,2157,2.96,0.96
2,203507,3,Giannis Antetokounmpo,1610612749,MIL,63,2203,723,1176,0.615,33,113,0.292,461,696,0.662,156,550,706,404,78,63,213,181,1940,2290,1.90,0.37
3,1628369,4,Jayson Tatum,1610612738,BOS,62,2224,570,1206,0.473,194,518,0.375,343,413,0.831,55,463,518,303,61,35,157,125,1677,1731,1.93,0.39
4,1630162,5,Anthony Edwards,1610612750,MIN,63,2214,583,1260,0.463,156,425,0.367,344,411,0.837,37,296,333,317,80,35,196,119,1666,1491,1.62,0.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,1630311,537,Pat Spencer,1610612744,GSW,2,3,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,1,0,0,0,0,0,1,0.00,0.00
550,1631250,537,Pete Nance,1610612739,CLE,3,7,0,2,0.000,0,0,0.000,0,0,0.000,0,1,1,0,0,0,0,0,0,-1,0.00,0.00
551,1631199,537,Ron Harper Jr.,1610612761,TOR,1,4,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,1,0,0,0,2,0,1,0.00,0.00
552,1627853,537,Ryan Arcidiacono,1610612752,NYK,20,45,0,6,0.000,0,6,0.000,0,0,0.000,0,8,8,4,2,0,2,5,0,6,2.00,1.00


In [ ]:
df_cols = ['Year'] + table_headers

In [ ]:
pd.DataFrame(columns=df_cols)

,Year,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV


In [ ]:
headers = {'Accept': '*/*',
  'Accept-Encoding':'gzip, deflate, br',
  'Accept-Language': 'pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7',
  'Connection': 'keep-alive',
  'Dnt': '1',
  'Host': 'stats.nba.com',
  'Origin': 'https://www.nba.com',
  'Referer': 'https://www.nba.com/',
  'Sec-Ch-Ua': '"Not(A:Brand";v="24", "Chromium";v="122"',
  'Sec-Ch-Ua-Mobile': '?0',
  'Sec-Ch-Ua-Platform': '"macOS"',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-site',
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

In [ ]:
df = pd.DataFrame(columns=df_cols)
years = ['2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']

begin_loop = time.time()

for y in years:
  api_url = 'https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=Totals&Scope=S&Season='+y+'&SeasonType=Regular%20Season&StatCategory=PTS'
  r = requests.get(url = api_url, headers=headers).json()
  temp_df1 = pd.DataFrame(r['resultSet']['rowSet'], columns=table_headers)
  temp_df2 = pd.DataFrame({'Year':[y for i in range(len(temp_df1))]})
  temp_df3 = pd.concat([temp_df2, temp_df1], axis = 1)
  df = pd.concat([df, temp_df3], axis=0)
  print(f'Finished scraping data for the {y} season.')
  lag = np.random.uniform(low=10,high=30)
  print(f'...wainting {round(lag, 1)} seconds')
  time.sleep(lag)

print(f'Process completed! Total run time: {round((time.time()-begin_loop)/60, 2)} minutes')

Finished scraping data for the 2013-14 season.
...wainting 28.8 seconds
Finished scraping data for the 2014-15 season.
...wainting 17.7 seconds
Finished scraping data for the 2015-16 season.
...wainting 11.8 seconds
Finished scraping data for the 2016-17 season.
...wainting 13.5 seconds
Finished scraping data for the 2017-18 season.
...wainting 28.7 seconds
Finished scraping data for the 2018-19 season.
...wainting 11.3 seconds
Finished scraping data for the 2019-20 season.
...wainting 15.2 seconds
Finished scraping data for the 2020-21 season.
...wainting 22.1 seconds
Finished scraping data for the 2021-22 season.
...wainting 24.3 seconds
Finished scraping data for the 2022-23 season.
...wainting 23.9 seconds
Finished scraping data for the 2023-24 season.
...wainting 29.0 seconds
Process completed! Total run time: 3.88 minutes


In [ ]:
df.to_excel('nba_player_2013-2024_data.xlsx', index=False)

# Data Analysis

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Licencjat/nba_player_2013-2024_data.xlsx')
data

,Year,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV
0,2013-14,201142,1,Kevin Durant,1610612760,OKC,81,3122,849,1688,0.503,192,491,0.391,703,805,0.873,58,540,598,445,103,59,285,174,2593,2572,1.56,0.36
1,2013-14,2546,2,Carmelo Anthony,1610612752,NYK,77,2982,743,1643,0.452,167,415,0.402,459,541,0.848,145,477,622,242,95,51,198,224,2112,1942,1.22,0.48
2,2013-14,2544,3,LeBron James,1610612748,MIA,77,2902,767,1353,0.567,116,306,0.379,439,585,0.750,81,452,533,488,121,26,270,126,2089,2255,1.81,0.45
3,2013-14,201567,4,Kevin Love,1610612750,MIN,77,2797,650,1421,0.457,190,505,0.376,520,633,0.821,224,739,963,341,59,35,196,136,2010,2328,1.74,0.30
4,2013-14,201933,5,Blake Griffin,1610612746,LAC,80,2863,718,1359,0.528,12,44,0.273,482,674,0.715,192,565,757,309,92,51,224,265,1930,2082,1.38,0.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5768,2023-24,1630311,537,Pat Spencer,1610612744,GSW,2,3,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,1,0,0,0,0,0,1,0.00,0.00
5769,2023-24,1631250,537,Pete Nance,1610612739,CLE,3,7,0,2,0.000,0,0,0.000,0,0,0.000,0,1,1,0,0,0,0,0,0,-1,0.00,0.00
5770,2023-24,1631199,537,Ron Harper Jr.,1610612761,TOR,1,4,0,0,0.000,0,0,0.000,0,0,0.000,0,0,0,1,0,0,0,2,0,1,0.00,0.00
5771,2023-24,1627853,537,Ryan Arcidiacono,1610612752,NYK,20,45,0,6,0.000,0,6,0.000,0,0,0.000,0,8,8,4,2,0,2,5,0,6,2.00,1.00


In [ ]:
years = ['2013-14','2014-2015', '2015-2016', '2016-2017', '2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023', '2022-2023']
#salary_data = pd.DataFrame(columns=["Player", "Salary", "Season"])
def get_salary(y):
  season=y
  url=f'https://hoopshype.com/salaries/players/'+season+'/'
  soup=BeautifulSoup(urlopen(url),'html.parser')
  salary=pd.read_html(str(soup))[0]
  salary=salary.iloc[:,[1,3]]
  salary=salary.rename(columns={salary.columns[1]:'Salary'})
  return salary



In [ ]:
get_salary('2013-14')

,Player,Salary
0,Stephen Curry,"$55,761,217"
1,Kevin Durant,"$51,179,020"
2,Nikola Jokic,"$51,415,938"
3,LeBron James,"$51,415,938"
4,Joel Embiid,"$51,415,938"
...,...,...
601,Taze Moore,$0
602,Taevion Kinsey,$0
603,Edmond Sumner,$0
604,Kaiser Gates,$0


**Data cleaning**

In [ ]:
data.drop(columns=["RANK","EFF"], inplace=True)

In [ ]:
data['season'] = data['Year'].str[:4].astype(int) + 1

In [ ]:
data['TEAM'].replace(to_replace=['NOP','NOH'], value='NO', inplace=True)

In [ ]:
total_cols = ['MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

**Data analysis**

In [ ]:
data_per_min = data.groupby(['PLAYER', 'PLAYER_ID', 'Year'])[total_cols].sum().reset_index()
for col in data_per_min.columns[4:]:
  data_per_min[col] = data_per_min[col]/data_per_min['MIN']

data_per_min['FG%'] = data_per_min['FGM']/data_per_min['FGA']
data_per_min['3PT%'] = data_per_min['FG3M']/data_per_min['FG3A']
data_per_min['FT%'] = data_per_min['FTM']/data_per_min['FTA']
data_per_min['AST%'] = data_per_min['AST']/data_per_min['FGM']
data_per_min['FG3A%'] = data_per_min['FG3A']/data_per_min['FGA']
data_per_min['PTS/FGA'] = data_per_min['PTS']/data_per_min['FGA']
data_per_min['FG3M/FGM'] = data_per_min['FG3M']/data_per_min['FGM']
data_per_min['TS%'] = 0.5*data_per_min['PTS']/(data_per_min['FGA']+0.475*data_per_min['FTA'])
data_per_min['AST_TOV'] = data_per_min['AST']/data_per_min['TOV']

data_per_min = data_per_min[data_per_min['MIN']>=50]
data_per_min.drop(columns='PLAYER_ID', inplace=True)

fig = px.imshow(data_per_min.corr())
fig.show()

<ipython-input-17-2ae305e8f1db>:18: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [ ]:
data_per_min.sample(10)

,PLAYER,Year,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,STL,BLK,TOV,PF,PTS,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,TS%,AST_TOV
4194,Nerlens Noel,2018-19,1056,0.153409,0.261364,0.000000,0.000000,0.051136,0.074811,0.120265,0.187500,0.042614,0.062500,0.090909,0.045455,0.157197,0.357955,0.586957,NaN,0.683544,0.277778,0.000000,1.369565,0.000000,0.602823,0.937500
5582,Vince Carter,2015-16,1005,0.132338,0.341294,0.058706,0.168159,0.069652,0.083582,0.030846,0.114428,0.055721,0.037811,0.015920,0.034826,0.100498,0.393035,0.387755,0.349112,0.833333,0.421053,0.492711,1.151603,0.443609,0.515800,1.600000
634,Buddy Hield,2021-22,2499,0.178471,0.439376,0.104842,0.286515,0.044418,0.050820,0.026411,0.114846,0.091637,0.029212,0.010004,0.061224,0.073629,0.506202,0.406193,0.365922,0.874016,0.513453,0.652095,1.152095,0.587444,0.546047,1.496732
2663,Jericho Sims,2021-22,555,0.070270,0.097297,0.000000,0.000000,0.021622,0.052252,0.109910,0.194595,0.037838,0.019820,0.039640,0.036036,0.118919,0.162162,0.722222,NaN,0.413793,0.538462,0.000000,1.666667,0.000000,0.663962,1.050000
3053,Justin Anderson,2019-20,107,0.093458,0.355140,0.056075,0.271028,0.018692,0.037383,0.009346,0.186916,0.074766,0.000000,0.056075,0.037383,0.121495,0.261682,0.263158,0.206897,0.500000,0.800000,0.763158,0.736842,0.600000,0.350877,2.000000
4139,Naji Marshall,2023-24,1007,0.135055,0.293942,0.046673,0.121152,0.064548,0.081430,0.047666,0.143992,0.107249,0.037736,0.010924,0.056604,0.072493,0.381331,0.459459,0.385246,0.792683,0.794118,0.412162,1.297297,0.345588,0.573220,1.894737
1586,Dwight Howard,2021-22,971,0.138002,0.225541,0.008239,0.015448,0.098867,0.150360,0.121524,0.245108,0.036045,0.035015,0.037075,0.048404,0.118435,0.383110,0.611872,0.533333,0.657534,0.261194,0.068493,1.698630,0.059701,0.645049,0.744681
242,Andrew Bogut,2014-15,1583,0.126342,0.224258,0.000000,0.000000,0.013898,0.026532,0.089071,0.253948,0.113708,0.024637,0.071383,0.066961,0.118762,0.266582,0.563380,NaN,0.523810,0.900000,0.000000,1.188732,0.000000,0.562742,1.698113
5433,Troy Brown Jr.,2021-22,1055,0.102370,0.244550,0.044550,0.126066,0.018957,0.024645,0.036967,0.155450,0.062559,0.034123,0.004739,0.024645,0.058768,0.268246,0.418605,0.353383,0.769231,0.611111,0.515504,1.096899,0.435185,0.523396,2.538462
4405,Pablo Prigioni,2015-16,823,0.066829,0.178615,0.034022,0.115431,0.008505,0.009721,0.046173,0.092345,0.157959,0.064399,0.001215,0.059538,0.099635,0.176185,0.374150,0.294737,0.875000,2.363636,0.646259,0.986395,0.509091,0.480769,2.653061


In [ ]:
data_per_36 = data_per_min
data_per_36[total_cols] = data_per_36[total_cols]*36
data_per_36['MIN'] = data_per_36['MIN']/36


In [ ]:
data_per_36.sort_values('PTS', ascending=False)

,PLAYER,Year,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,STL,BLK,TOV,PF,PTS,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,TS%,AST_TOV
2765,Joel Embiid,2023-24,1157.0,12.508211,23.460674,1.275713,3.484875,11.076923,12.539326,2.769231,9.178911,6.005186,1.213483,1.866897,3.920484,3.049265,37.369058,0.533156,0.366071,0.883375,0.480100,0.148541,1.592838,0.101990,0.635164,1.531746
2398,James Harden,2018-19,2867.0,10.585281,23.970701,4.746425,12.908266,9.467736,10.773631,0.828741,5.675619,7.358214,1.983955,0.728287,4.859435,3.063830,35.384723,0.441592,0.367704,0.878788,0.695136,0.538502,1.476166,0.448399,0.608232,1.514212
1908,Giannis Antetokounmpo,2019-20,1917.0,12.863850,23.248826,1.671362,5.502347,7.474178,11.812207,2.629108,13.446009,6.647887,1.145540,1.239437,4.319249,3.661972,34.873239,0.553312,0.303754,0.632750,0.516788,0.236672,1.500000,0.129927,0.604187,1.539130
1911,Giannis Antetokounmpo,2022-23,2024.0,12.575099,22.731225,0.835968,3.041502,8.857708,13.731225,2.436759,10.760870,6.385375,0.924901,0.907115,4.375494,3.503953,34.843874,0.553208,0.274854,0.645078,0.507779,0.133803,1.532864,0.066478,0.595549,1.459350
2764,Joel Embiid,2022-23,2284.0,11.474606,20.931699,1.040280,3.152364,10.418564,12.152364,1.781086,8.779335,4.318739,1.040280,1.765324,3.562172,3.231173,34.408056,0.548193,0.330000,0.857328,0.376374,0.150602,1.643825,0.090659,0.644247,1.212389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2454,Jared Dudley,2020-21,81.0,0.888889,4.000000,0.888889,2.666667,0.000000,0.000000,1.777778,7.555556,2.222222,0.444444,0.444444,0.888889,3.111111,2.666667,0.222222,0.333333,NaN,2.500000,0.666667,0.666667,1.000000,0.333333,2.500000
2132,Isaiah Taylor,2016-17,52.0,0.692308,4.846154,0.000000,1.384615,0.692308,1.384615,0.692308,1.384615,2.076923,0.692308,0.692308,0.000000,4.153846,2.076923,0.142857,0.000000,0.500000,3.000000,0.285714,0.428571,0.000000,0.188679,inf
5016,Solomon Hill,2021-22,139.0,0.776978,5.179856,0.517986,3.366906,0.000000,0.000000,1.553957,4.402878,3.107914,1.035971,0.517986,0.258993,4.143885,2.071942,0.150000,0.153846,NaN,4.000000,0.650000,0.400000,0.666667,0.200000,12.000000
1420,Devon Dotson,2022-23,53.0,0.679245,6.792453,0.679245,2.716981,0.000000,0.000000,4.075472,2.716981,5.433962,3.396226,0.000000,2.037736,6.113208,2.037736,0.100000,0.250000,NaN,8.000000,0.400000,0.300000,1.000000,0.150000,2.666667


**How are minutes played distributed?**

In [ ]:
fig = px.histogram(x=df['MIN'])
fig.show()

**How has the game changed over the past 10 years?**

In [ ]:
change_df = data.groupby('season_start_year')[total_cols].sum().reset_index()
change_df['est_POSS'] = change_df['FGA'] - change_df['OREB'] + change_df['TOV']+0.44*change_df['FTA']
change_df = change_df[list(change_df.columns[0:2])+['est_POSS']+list(change_df.columns[2:-1])]

change_df['FG%'] = change_df['FGM']/change_df['FGA']
change_df['3PT%'] = change_df['FG3M']/change_df['FG3A']
change_df['FT%'] = change_df['FTM']/change_df['FTA']
change_df['AST%'] = change_df['AST']/change_df['FGM']
change_df['FG3A%'] = change_df['FG3A']/change_df['FGA']
change_df['PTS/FGA'] = change_df['PTS']/change_df['FGA']
change_df['FG3M/FGM'] = change_df['FG3M']/change_df['FGM']
change_df['TS%'] = 0.5*change_df['PTS']/(change_df['FGA']+0.475*change_df['FTA'])
change_df['AST_TOV'] = change_df['AST']/change_df['TOV']

change_df

,season_start_year,MIN,est_POSS,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,AST,STL,BLK,TOV,PF,PTS,FG%,3PT%,FT%,AST%,FG3A%,PTS/FGA,FG3M/FGM,TS%,AST_TOV
0,2013,595202,237424.76,92779,204172,19054,52974,43870,58029,26846,78315,54131,18895,11594,34566,50923,248482,0.454416,0.359686,0.756001,0.583440,0.259458,1.217023,0.205370,0.536132,1.566019
1,2014,595214,237241.12,92287,205570,19300,55137,42161,56198,26781,79723,54202,19031,11797,33725,49728,246035,0.448932,0.350037,0.750222,0.587320,0.268215,1.196843,0.209130,0.529645,1.607176
2,2015,594864,241647.36,94065,208049,20953,59241,43489,57469,25624,82021,54831,19303,12193,33936,49854,252572,0.452129,0.353691,0.756738,0.582905,0.284745,1.214002,0.222750,0.536595,1.615718
3,2016,594409,243091.20,96061,210114,23748,66421,43883,56855,24936,82109,55660,18950,11669,32897,48950,259753,0.457185,0.357538,0.771841,0.579423,0.316119,1.236248,0.247218,0.547724,1.691948
4,2017,593865,244918.00,97435,211707,25807,71339,40903,53325,23890,83159,57163,18983,11845,33638,48837,261580,0.460235,0.361752,0.767051,0.586678,0.336970,1.235576,0.264864,0.551772,1.699358
5,2018,594465,252270.52,101062,219458,27955,78742,43494,56758,25454,85653,60483,18779,12185,33293,51425,273573,0.460507,0.355020,0.766306,0.598474,0.358802,1.246585,0.276612,0.555099,1.816688
6,2019,512068,217780.92,86550,188116,25862,72252,37826,48943,21340,73617,51649,16200,10378,29470,44004,236788,0.460088,0.357942,0.772858,0.596753,0.384082,1.258734,0.298810,0.560143,1.752596
7,2020,521512,219058.40,89020,190983,27427,74822,36650,47135,21232,74454,53577,16356,10524,28568,41669,242117,0.466115,0.366563,0.777554,0.601854,0.391773,1.267741,0.308099,0.567359,1.875420
8,2021,593758,247080.64,99930,216722,30598,86535,41657,53781,25422,83925,60636,18772,11594,32117,48306,272115,0.461098,0.353591,0.774567,0.606785,0.399290,1.255595,0.306194,0.561599,1.887972
9,2022,594863,250057.64,103260,217219,30359,84165,45248,57881,25669,81189,62279,17933,11448,33040,49149,282127,0.475373,0.360708,0.781742,0.603128,0.387466,1.298814,0.294005,0.576446,1.884958


In [ ]:
change_per48_df = change_df.copy()
for col in change_per48_df.columns[2:17]:
  change_per48_df[col] = (change_per48_df[col]/change_per48_df['MIN'])*48*5

change_per48_df.drop(columns='MIN', inplace=True)
change_per48_df
fig = go.Figure()
for col in change_per48_df.columns[1:]:
 fig.add_trace(go.Scatter(x=change_per48_df['season_start_year'], y=change_per48_df[col], name=col))

fig.show()

In [ ]:
change_per100_df = change_df.copy()
for col in change_per100_df.columns[3:18]:
  change_per100_df[col] = (change_per100_df[col]/change_per100_df['est_POSS'])*100

change_per100_df.drop(columns=['MIN', 'est_POSS'], inplace=True)

fig = go.Figure()
for col in change_per100_df.columns[1:]:
  fig.add_trace(go.Scatter(x=change_per100_df['season_start_year'], y=change_per100_df[col], name=col))

fig.show()